In [2]:
from langgraph.graph import StateGraph, END
from langgraph.types import Command, interrupt
from typing import TypedDict, Annotated, List
from langgraph.checkpoint.memory import MemorySaver
import operator
memory = MemorySaver()

class AgentState(TypedDict):
    value: Annotated[str, operator.add]

def node_a(state: AgentState):
    print("Node A")
    return Command(
        goto="node_b",
        update= { "value": "A"}
    )

def node_b(state: AgentState):
    print("Node B")

    human_reponse = interrupt("Do you want to goto node C/D")
    print("Human Review Values: ", human_reponse)

    if human_reponse=="C":
        return Command(
            goto="node_c",
            update={"value": "B"}
        )
    
    if human_reponse=="D":
        return Command(
            goto="node_d",
            update={"value": "B"}
        )
    
def node_c(state: AgentState):
    print("Node C")
    return Command(
        goto=END,
        update= { "value": "C"}
    )

def node_d(state: AgentState):
    print("Node D")
    return Command(
        goto=END,
        update= { "value": "D"}
    )

graph = StateGraph(AgentState)

graph.add_node("node_a", node_a)
graph.add_node("node_b", node_b)
graph.add_node("node_c", node_c)
graph.add_node("node_d", node_d)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)
config = {"configurable": {"thread_id": 1}}

In [7]:
value = {"value": ""}
initial_result = app.invoke(value, config=config, stream_mode="updates")
initial_result

Node A
Node B


[{'node_a': {'value': 'a'}},
 {'__interrupt__': (Interrupt(value='Do you want to go to C or D? Type C/D', id='c7138dbceaab12a9a90a27efd72f1e08'),)}]

In [8]:
#Printing the resume node
print(app.get_state(config).next)

# Use Command(resume=...) to resume from the interrupted method
final_result = app.invoke(Command(resume='C'), config=config, stream_mode="updates")

('node_b',)
Node B
Human Review Values:  C
Node C
